<a href="https://colab.research.google.com/github/itsMeMuskanKumari/-Hand_Written_Digit_Classification.ipynb/blob/main/hackathon_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

orders = pd.read_csv("/orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [20]:
users = pd.read_json("/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [24]:
import sqlite3

conn = sqlite3.connect("restaurants.db")

with open("/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


Merge Orders + Users


In [25]:
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)

Merge with Restaurants

In [26]:
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

In [27]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


1. Order Trends Over Time

In [29]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

orders_by_date = final_df.groupby(final_df['order_date'].dt.month).size()


/tmp/ipython-input-2955595483.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


2. User Behavior Patterns

In [30]:
final_df.groupby("user_id")["order_id"].count().sort_values(ascending=False)


,order_id
user_id,
2973,13
1515,12
496,11
874,11
1337,11
...,...
2961,1
2957,1
2951,1


In [34]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

3. City-wise Performance

In [36]:
final_df.groupby("city")["total_amount"].sum()



,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Hyderabad,1889366.58
Pune,1924797.93


4. Cuisine-wise Performance

In [37]:
final_df.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


5. Membership Impact (Gold vs Regular)

In [38]:
final_df.groupby("membership")["total_amount"].agg(["count", "mean", "sum"])


,count,mean,sum
membership,,,
Gold,4987,797.145556,3975364.89
Regular,5013,805.158434,4036259.23


6. Revenue Distribution

In [39]:
final_df["total_amount"].describe()


,total_amount
count,10000.000000
mean,801.162412
std,405.458753
min,100.200000
25%,446.310000
50%,806.295000
75%,1149.227500
max,1499.830000


In [40]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")
df["order_date"] = pd.to_datetime(df["order_date"])


/tmp/ipython-input-3796389500.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


City with highest total revenue from Gold members

In [41]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


2️⃣ Cuisine with highest average order value

In [42]:
df.groupby("cuisine")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


Distinct users with total spend > 1000

In [43]:
high_value_users = (
    df.groupby("user_id")["total_amount"].sum() > 1000
).sum()

high_value_users


np.int64(2544)

In [44]:
user_total_spend = df.groupby("user_id")["total_amount"].sum()


In [45]:
exact_count = (user_total_spend > 1000).sum()
exact_count


np.int64(2544)

Rating range with highest revenue

In [48]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-125571628.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


Highest average order value among Gold members (by city)

In [49]:
df[df["membership"] == "Gold"] \
  .groupby("city")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


Fewest restaurants but strong revenue (by cuisine)

In [50]:
df.groupby("cuisine").agg(
    restaurants=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
).sort_values(["restaurants", "revenue"], ascending=[True, False])


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


% of orders by Gold members

In [51]:
round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)


50

Highest avg order value, < 20 orders

In [53]:
df.groupby("restaurant_name_x").agg(
    orders=("order_id", "count"),
    avg_value=("total_amount", "mean")
).query("orders < 20").sort_values("avg_value", ascending=False)


,orders,avg_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [54]:
df.groupby("restaurant_name_y").agg(
    orders=("order_id", "count"),
    avg_value=("total_amount", "mean")
).query("orders < 20").sort_values("avg_value", ascending=False)


,orders,avg_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


Highest revenue combination

In [55]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

Highest revenue quarter

In [56]:
df["quarter"] = df["order_date"].dt.quarter

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")


df['order_date'] = pd.to_datetime(df['order_date'])


Total orders placed by Gold members

In [57]:
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
gold_orders_count


4987

Total revenue generated from orders placed in Hyderabad (nearest integer)

In [58]:
hyderabad_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())
hyderabad_revenue


1889367

Number of distinct users who placed at least one order

In [59]:
distinct_users = df['user_id'].nunique()
distinct_users


2883

Average order value for Gold members (2 decimals)

In [60]:
avg_gold_order_value = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)
avg_gold_order_value


np.float64(797.15)

Orders placed for restaurants with rating ≥ 4.5

In [61]:
high_rating_orders = df[df['rating'] >= 4.5].shape[0]
high_rating_orders


3374

Orders placed in the top revenue city among Gold members only

In [63]:
# Total revenue per city for Gold members
gold_city_revenue = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()

# Top revenue city
top_gold_city = gold_city_revenue.idxmax()

# Orders in that city
orders_top_gold_city = df[(df['membership'] == 'Gold') & (df['city'] == top_gold_city)].shape[0]
orders_top_gold_city


1337